<a href="https://colab.research.google.com/github/daniel-men/transformerNotebookColab/blob/main/torch_io_tutorial_for_transf3d_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## self-attention-cv : illustration of a training process with subvolume sampling for 3d segmentation

The dataset can be found here: https://iseg2019.web.unc.edu/ . i uploaded it and mounted from my gdrive

In [4]:
from google.colab import drive
drive.mount('/gdrive')
import tarfile
root_path = '/gdrive/My Drive/Task01_BrainTumour.tar' 
!echo "Download and extracting folders..."
zip_ref = tarfile.TarFile(root_path, 'r')
zip_ref.extractall("./")
zip_ref.close()
!echo "Finished"
!pip install torchio
!pip install self-attention-cv

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
Download and extracting folders...
Finished
     |████████████████████████████████| 143kB 19.0MB/s 
     |████████████████████████████████| 42.5MB 63kB/s 
     |████████████████████████████████| 286kB 18.4MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Found existing installation: pytest 3.6.4
    Uninstalling pytest-3.6.4:
      Successfully uninstalled pytest-3.6.4


In [8]:
!ls ./Task01_BrainTumour
!ls ./Task01_BrainTumour/imagesTr


dataset.json  imagesTr	imagesTs  labelsTr
BRATS_001.nii.gz  BRATS_122.nii.gz  BRATS_243.nii.gz  BRATS_364.nii.gz
BRATS_002.nii.gz  BRATS_123.nii.gz  BRATS_244.nii.gz  BRATS_365.nii.gz
BRATS_003.nii.gz  BRATS_124.nii.gz  BRATS_245.nii.gz  BRATS_366.nii.gz
BRATS_004.nii.gz  BRATS_125.nii.gz  BRATS_246.nii.gz  BRATS_367.nii.gz
BRATS_005.nii.gz  BRATS_126.nii.gz  BRATS_247.nii.gz  BRATS_368.nii.gz
BRATS_006.nii.gz  BRATS_127.nii.gz  BRATS_248.nii.gz  BRATS_369.nii.gz
BRATS_007.nii.gz  BRATS_128.nii.gz  BRATS_249.nii.gz  BRATS_370.nii.gz
BRATS_008.nii.gz  BRATS_129.nii.gz  BRATS_250.nii.gz  BRATS_371.nii.gz
BRATS_009.nii.gz  BRATS_130.nii.gz  BRATS_251.nii.gz  BRATS_372.nii.gz
BRATS_010.nii.gz  BRATS_131.nii.gz  BRATS_252.nii.gz  BRATS_373.nii.gz
BRATS_011.nii.gz  BRATS_132.nii.gz  BRATS_253.nii.gz  BRATS_374.nii.gz
BRATS_012.nii.gz  BRATS_133.nii.gz  BRATS_254.nii.gz  BRATS_375.nii.gz
BRATS_013.nii.gz  BRATS_134.nii.gz  BRATS_255.nii.gz  BRATS_376.nii.gz
BRATS_014.nii.gz  BRATS_135.nii.gz 

## Training example

In [1]:
import glob
import torchio as tio
import torch
from torch.utils.data import DataLoader

paths_t1 = sorted(glob.glob('./Task01_BrainTumour/imagesTr/*.nii.gz'))[:50]
#paths_t2 = sorted(glob.glob('./iSeg-2019-Training/*T2.img'))
paths_seg = sorted(glob.glob('./Task01_BrainTumour/labelsTr/*.nii.gz'))[:50]
assert len(paths_t1) == len(paths_seg)

subject_list = []
for pat in zip(paths_t1, paths_seg):
  path_t1, path_seg = pat
  image = tio.ScalarImage(path_t1,)
  t1 = tio.ScalarImage(tensor=image.data[:, :, :, 0].unsqueeze(0))
  t2 = tio.ScalarImage(tensor=image.data[:, :, :, 2].unsqueeze(0))
  subject = tio.Subject(t1=t1, t2=t2, label=tio.LabelMap(path_seg)) 
  subject_list.append(subject)


transforms = [tio.RescaleIntensity((0, 1)),tio.RandomAffine() ]
transform = tio.Compose(transforms)

subjects_dataset = tio.SubjectsDataset(subject_list, transform=transform)

patch_size = 24
queue_length = 300
samples_per_volume = 50
sampler = tio.data.UniformSampler(patch_size)

patches_queue = tio.Queue(
subjects_dataset,
queue_length,
samples_per_volume,sampler, num_workers=1)

patches_loader = DataLoader(patches_queue, batch_size=16)

In [ ]:
from self_attention_cv.Transformer3Dsegmentation import Transformer3dSeg

def crop_target(img, target_size):
  dim = img.shape[-1]
  center = dim//2
  start_dim = center - (target_size//2) - 1
  end_dim = center + (target_size//2)
  return img[:,0,start_dim:end_dim,start_dim:end_dim,start_dim:end_dim].long()

target_size = 3 # as in the paper 
patch_dim = 8
num_epochs = 50
num_classes = 4
model = Transformer3dSeg(subvol_dim=patch_size, patch_dim=patch_dim,
                         in_channels=2, blocks=2, num_classes=num_classes).cuda()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print(len(patches_loader))
for epoch_index in range(num_epochs):
  epoch_loss = 0
  for c,patches_batch in enumerate(patches_loader):
    optimizer.zero_grad()
    
    input_t1 = patches_batch['t1'][tio.DATA]  
    input_t2 = patches_batch['t2'][tio.DATA]

    input_tensor = torch.cat([input_t1, input_t2], dim=1).cuda()
    
    
    logits = model(input_tensor) # 8x8x8 the 3d transformer-based approach

    # for the 3d transformer-based approach the target must be cropped again to the desired size
    targets = patches_batch['label'][tio.DATA]  
    
    cropped_target = crop_target(targets, target_size).cuda()

    loss = criterion(logits, cropped_target)
    loss.backward()
    optimizer.step()
    epoch_loss = epoch_loss+loss.cpu().item()

  print(f'epoch {epoch_index} loss {epoch_loss/c}')
    



157
epoch 0 loss 0.19215610983948678
epoch 1 loss 0.08945363417912561
epoch 2 loss 0.09652531164913224
epoch 3 loss 0.09292837228172292
epoch 4 loss 0.09473018650299846
epoch 5 loss 0.0801666445003297
epoch 6 loss 0.09511692491240609


## Inference

In [ ]:
import torch
import torch.nn as nn
import torchio as tio
patch_overlap = 0
patch_size = 24, 24, 24
target_patch_size = 3

#input sampling
grid_sampler = tio.inference.GridSampler(subject_list[0], patch_size, patch_overlap)
patch_loader = torch.utils.data.DataLoader(grid_sampler, batch_size=4)
# target vol sampling
grid_sampler_target = tio.inference.GridSampler(subject_list[0], target_patch_size, patch_overlap)
aggregator = tio.inference.GridAggregator(grid_sampler_target)
target_loader = torch.utils.data.DataLoader(grid_sampler_target, batch_size=4)

model.eval()

with torch.no_grad():
  for patches_batch,target_patches in zip(patch_loader,target_loader):

    input_t1 = patches_batch['t1'][tio.DATA]  
    input_t2 = patches_batch['t2'][tio.DATA]
    input_tensor = torch.cat([input_t1, input_t2], dim=1).float().cuda()

    locations = target_patches[tio.LOCATION]
    logits = model(input_tensor)
    labels = logits.argmax(dim=tio.CHANNELS_DIMENSION, keepdim=True)
    outputs = labels
    aggregator.add_batch(outputs.type(torch.int32), locations)

  print('output tensor shape:',outputs.shape)
  output_tensor = aggregator.get_output_tensor()
  print(output_tensor.shape)

output tensor shape: torch.Size([4, 1, 3, 3, 3])
torch.Size([1, 144, 192, 256])
